In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/neuralcoref-master

/content/drive/MyDrive/neuralcoref-master


In [3]:
!pwd
!pip install -r ./neuralcoref/train/training_requirements.txt -e .
!python -m spacy download en

/content/drive/MyDrive/neuralcoref-master
Obtaining file:///content/drive/MyDrive/neuralcoref-master
     |████████████████████████████████| 734.6MB 23kB/s 
     |████████████████████████████████| 122kB 46.4MB/s 
     |████████████████████████████████| 133kB 44.6MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 7.6MB 32.5MB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.3.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.3.1 which is incompatible.
ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Running setup.py develop for neuralcoref
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking su

In [4]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=e379902c3472f09bb56c4b23be4e8d8d11ef8368869250629d445cc576192a6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-obo4wn5t/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [5]:
import sys
import pandas as pd
import numpy as np
import spacy
import re
import torch
import neuralcoref
import en_core_web_lg

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:04<00:00, 9330963.74B/s]


In [6]:
# samo promjenite path ako stavite podatke na drugo mjesto i ne znam zasto, al kad se makne ovaj name argument nece ucitat podatke, pa ako se nekom da otkrit zasto nek proba
unprocessed_data = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/friends_data/friends.train.episode_delim.conll", sep=" ", names=["Document ID"])


def load_data_fn(data, ) -> pd.DataFrame:
  """
  Rjesava se praznina koje dobijemo kod obicnog pd.read_csv

  PARAMETERS:
    data (pd.Dataframe): pandas dataframe kad ucitas sa pd.read_csv
  """
  with open("/content/drive/MyDrive/friends_data/friends.train.episode_delim.conll") as d:
    result = []
    for item in d:
      tmp = re.sub("\s", "|", item)
      tmp2 = tmp.split("|")
      lst = []
      for el in tmp2:
        if el != "":
          lst.append(el)
      result.append(lst)
    df = pd.DataFrame(result, columns=["Document ID","Scene ID","Token ID","Word form","Part-of-speech tag","Constituency tag","Lemma","Frameset ID", "Word sense", "Speaker", "Named entity tag", "Entity ID"])
    df.drop(df.tail(1).index,inplace=True)
    df.drop(df.head(1).index,inplace=True)
    df.dropna(inplace=True)
    return df

data = load_data_fn(unprocessed_data)

In [7]:
data.head(20)
#data.dtypes

,Document ID,Scene ID,Token ID,Word form,Part-of-speech tag,Constituency tag,Lemma,Frameset ID,Word sense,Speaker,Named entity tag,Entity ID
1,/friends-s01e01,0,0,There,EX,(TOP(S(NP*),there,-,-,Monica_Geller,*,-
2,/friends-s01e01,0,1,'s,VBZ,(VP*,be,-,-,Monica_Geller,*,-
3,/friends-s01e01,0,2,nothing,NN,(NP*,nothing,-,-,Monica_Geller,*,-
4,/friends-s01e01,0,3,to,TO,(S(VP*,to,-,-,Monica_Geller,*,-
5,/friends-s01e01,0,4,tell,VB,(VP*))))),tell,-,-,Monica_Geller,*,-
6,/friends-s01e01,0,5,!,.,*)),!,-,-,Monica_Geller,*,-
8,/friends-s01e01,0,0,He,PRP,(TOP(S(NP*),he,-,-,Monica_Geller,*,(284)
9,/friends-s01e01,0,1,'s,VBZ,(VP*,be,-,-,Monica_Geller,*,-
10,/friends-s01e01,0,2,just,RB,(ADVP*),just,-,-,Monica_Geller,*,-
11,/friends-s01e01,0,3,some,DT,(NP(NP*,some,-,-,Monica_Geller,*,-


In [8]:
named_entity_map = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/friends_data/friends_entity_map.txt", sep="\t", names=['Entity ID', 'Person'])
named_entity_map.dtypes
df_named_entity = pd.DataFrame(named_entity_map, columns = ['Entity ID', 'Person'])

In [9]:
df = pd.DataFrame(data, columns = ["Document ID","Scene ID","Token ID","Word form","Part-of-speech tag","Constituency tag","Lemma","Frameset ID", "Word sense", "Speaker", "Named entity tag", "Entity ID"])
df['Document ID']

1         /friends-s01e01
2         /friends-s01e01
3         /friends-s01e01
4         /friends-s01e01
5         /friends-s01e01
               ...       
180019    /friends-s02e24
180020    /friends-s02e24
180021    /friends-s02e24
180022    /friends-s02e24
180023    /friends-s02e24
Name: Document ID, Length: 160171, dtype: object

In [10]:
#epizode
import random
print(df.loc[df['Token ID'] == '0'])

last_ep = '/friends-s01e01'
episodes = list()
sentences = ''
for index, row in df.iterrows():
  episode = str(row['Document ID'])
  if episode != last_ep:
    episodes.append(sentences)
    sentences = ''
    last_ep = episode

  word = row['Word form']
  if word[0] == '\'' or word[0] == '.' or word[0] == ',' or word[0] == '!' or word[0] == '?':
    sentences = sentences[:-1] + word + ' '
    continue


  id_string = str(row['Entity ID'])
  if id_string != '-':
    prob = random.random()
    if prob <= 0.6:
      zam = word.upper()
      if zam != 'I':
        if zam == 'HE' or zam == 'SHE' or zam == 'HER' or zam == 'HIM':
          if id_string[0] == '(':
            id_string = id_string[1:]
          if id_string[-1] == ')':
            id_string = id_string[:-1]
          sentences += df_named_entity.loc[df_named_entity['Entity ID'] == int(id_string), 'Person'].item() + ' '
          continue

  sentences += row['Word form'] + ' '
    
print(sentences)
for s in episodes:
  print(s)


            Document ID Scene ID  ... Named entity tag Entity ID
1       /friends-s01e01        0  ...                *         -
8       /friends-s01e01        0  ...                *     (284)
18      /friends-s01e01        0  ...                *         -
29      /friends-s01e01        0  ...                *         -
40      /friends-s01e01        0  ...                *         -
...                 ...      ...  ...              ...       ...
179942  /friends-s02e24        0  ...                *         -
179971  /friends-s02e24        0  ...                *         -
179983  /friends-s02e24        0  ...                *         -
180007  /friends-s02e24        0  ...                *         -
180017  /friends-s02e24        0  ...         (PERSON)         -

[19761 rows x 12 columns]
Hey Joey, how'd the audition go? Incredible! I met the director this time and you'll never believe who it was. Who? All right. I'll give you one hint. Warren Beatty. Wow! Yeah, there's just one

In [11]:
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

for s in episodes:
  doc2 = nlp(s)
  print(s)

  for ent in doc2.ents:
      print(ent._.coref_cluster)

Streaming output truncated to the last 5000 lines.
None
Paolo: [Paolo, Paolo, He, Paolo, Paolo, Paolo, Paolo, he, his, he, me]
Paolo: [Paolo, Paolo, He, Paolo, Paolo, Paolo, Paolo, he, his, he, me]
Monica: [Monica, Monica]
None
None
None
Paolo: [Paolo, Paolo, He, Paolo, Paolo, Paolo, Paolo, he, his, he, me]
None
None
Paolo: [Paolo, Paolo, He, Paolo, Paolo, Paolo, Paolo, he, his, he, me]
Rachel: [Rachel, him, him, He]
None
Paolo: [Paolo, Paolo, He, Paolo, Paolo, Paolo, Paolo, he, his, he, me]
Paolo: [Paolo, Paolo, He, Paolo, Paolo, Paolo, Paolo, he, his, he, me]
None
None
None
None
None
None
None
None
None
None
None
Rachel: [Rachel, Rachel]
Rachel: [Rachel, Rachel]
None
None
Chandler: [Chandler, Chandler, Chandler, Chandler, goodbye Chandler, Chandler]
None
None
None
None
None
None
None
None
Ross: [Ross, Ross, Ross, she, her, Ross]
Monica: [Monica, Monica]
Chandler: [Chandler, Chandler, Chandler, Chandler, goodbye Chandler, Chandler]
None
Dehydrated Japanese noodles under fluorescent li

In [14]:
# samo promjenite path ako stavite podatke na drugo mjesto i ne znam zasto, al kad se makne ovaj name argument nece ucitat podatke, pa ako se nekom da otkrit zasto nek proba
unprocessed_data = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/friends_data/friends.train.scene_delim.conll", sep=" ", names=["Document ID"])


def load_data_fn(data, ) -> pd.DataFrame:
  """
  Rjesava se praznina koje dobijemo kod obicnog pd.read_csv

  PARAMETERS:
    data (pd.Dataframe): pandas dataframe kad ucitas sa pd.read_csv
  """
  with open("/content/drive/MyDrive/friends_data/friends.train.scene_delim.conll") as d:
    result = []
    for item in d:
      tmp = re.sub("\s", "|", item)
      tmp2 = tmp.split("|")
      lst = []
      for el in tmp2:
        if el != "":
          lst.append(el)
      result.append(lst)
    df = pd.DataFrame(result, columns=["Document ID","Scene ID","Token ID","Word form","Part-of-speech tag","Constituency tag","Lemma","Frameset ID", "Word sense", "Speaker", "Named entity tag", "Entity ID"])
    df.drop(df.tail(1).index,inplace=True)
    df.drop(df.head(1).index,inplace=True)
    df.dropna(inplace=True)
    return df

data = load_data_fn(unprocessed_data)
df = pd.DataFrame(data, columns = ["Document ID","Scene ID","Token ID","Word form","Part-of-speech tag","Constituency tag","Lemma","Frameset ID", "Word sense", "Speaker", "Named entity tag", "Entity ID"])

In [ ]:
data.head(20)

In [ ]:
#scene
import random
print(df.loc[df['Token ID'] == '0'])

last_scene = '0'
scenes = list()
sentences = ''
for index, row in df.iterrows():
  scene = str(row['Scene ID'])
  if scene != last_scene:
    scenes.append(sentences)
    sentences = ''
    last_scene = scene

  word = row['Word form']
  if word[0] == '\'' or word[0] == '.' or word[0] == ',' or word[0] == '!' or word[0] == '?':
    sentences = sentences[:-1] + word + ' '
    continue

  id_string = str(row['Entity ID'])
  if id_string != '-':
    prob = random.random()
    if prob <= 0.6:
      zam = word.upper()
      if zam != 'I':
        if zam == 'HE' or zam == 'SHE' or zam == 'HER' or zam == 'HIM':
          if id_string[0] == '(':
            id_string = id_string[1:]
          if id_string[-1] == ')':
            id_string = id_string[:-1]
          sentences += df_named_entity.loc[df_named_entity['Entity ID'] == int(id_string), 'Person'].item() + ' '
          continue

  sentences += row['Word form'] + ' '

In [20]:
for s in scenes:
  print(s)

There's nothing to tell! Paul's just some guy I work with! C'mon, you're going out with the guy! There's got ta be something wrong with Paul! All right Joey, be nice. So does Paul have a hump? A hump and a hairpiece? Wait, does he eat chalk? Just,' cause, I do n't want her to go through what I went through with Carl - oh! Okay, everybody relax. This is not even a date. It's just two people going out to dinner and - not having sex. Sounds like a date to me. Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked. Oh, yeah. Had that dream. Then I look down, and I realize there's a phone... there. Instead of...? That's right. Never had that dream. No. All of a sudden, the phone starts to ring. Now I do n't know what to do, everybody starts looking at me. And they were n't looking at you before?! Finally, I figure I'd better answer it, and it turns out it's my mother, which is very - very weird, because - she never calls me! Hi. Th

In [21]:
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

for i in range(len(scenes)):
  print(i)
  doc2 = nlp(scenes[i])
  print(scenes[i])

  for ent in doc2.ents:
      print(ent._.coref_cluster)

0
There's nothing to tell! Paul's just some guy I work with! C'mon, you're going out with the guy! There's got ta be something wrong with Paul! All right Joey, be nice. So does Paul have a hump? A hump and a hairpiece? Wait, does he eat chalk? Just,' cause, I do n't want her to go through what I went through with Carl - oh! Okay, everybody relax. This is not even a date. It's just two people going out to dinner and - not having sex. Sounds like a date to me. Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked. Oh, yeah. Had that dream. Then I look down, and I realize there's a phone... there. Instead of...? That's right. Never had that dream. No. All of a sudden, the phone starts to ring. Now I do n't know what to do, everybody starts looking at me. And they were n't looking at you before?! Finally, I figure I'd better answer it, and it turns out it's my mother, which is very - very weird, because - she never calls me! Hi. 

KeyboardInterrupt: ignored

In [31]:
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

sentence = "Ben is talking to Mike. He asks him how is he"

doc2 = nlp(sentence)
print(sentence)

for ent in doc2.ents:
  print(ent._.coref_cluster)

Ben is talking to Mike. He asks him how is he
Ben: [Ben, He, him, he]
None
